In [276]:
# import matplotlib.pyplot as plt
import numpy as np
import glob
import os
import copy
# import pandas as pd

import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
from SeparableConv import SeparableConv1d

In [277]:
device = "mps" if torch.backends.mps.is_available() else "cpu"
device

'cpu'

## Dataset

In [278]:
class EarthquakeData(Dataset):
    def __init__(self, h_path, d_path):
        self.c_path = h_path + d_path
        self.h_len = len(h_path)
    
    def __len__(self):
        return len(self.c_path)
    
    def __getitem__(self, idx):
        path = self.c_path[idx]

        if idx > self.h_len:
            y = 1
        else:
            y = 0
            
        X = np.loadtxt(path, delimiter=',', dtype=str).astype(np.float32)

        return X, y

In [279]:
class1_paths = glob.glob("./128days/nonSSE/*.csv")
class2_paths = glob.glob("./128days/SSE/*.csv")

In [280]:
dataset = EarthquakeData(class1_paths, class2_paths)

In [281]:
dataset[0][0].shape

(30, 128)

In [282]:
# Calculat size for each dataloader (60% training, 20% validation and testing)
total_len = len(dataset)
train_size = int(total_len * 0.6)
val_size = int(total_len * 0.2)
test_size = total_len - train_size - val_size

train_data, val_data, test_data = torch.utils.data.random_split(dataset, [train_size, val_size, test_size])
print(train_size, val_size, test_size)

batch_size = 1
dataloader = DataLoader(train_data, batch_size, shuffle=True)
val_dataloader = DataLoader(val_data, batch_size, shuffle=False)
test_dataloader = DataLoader(test_data, batch_size, shuffle=False)
print(len(dataloader), len(test_dataloader), len(val_dataloader))

2151 717 717
2151 717 717


## Model

In [283]:
'''
https://github.com/s4rduk4r/eegnet_pytorch
    EEGNet PyTorch implementation
    Original implementation - https://github.com/vlawhern/arl-eegmodels
    Original paper: https://iopscience.iop.org/article/10.1088/1741-2552/aace8c

    ---
    EEGNet Parameters:

      nb_classes      : int, number of classes to classify
      Chans           : number of channels in the EEG data
      Samples         : sample frequency (Hz) in the EEG data
      dropoutRate     : dropout fraction
      kernLength      : length of temporal convolution in first layer. 
                        ARL recommends to set this parameter to be half of the sampling rate. 
                        For the SMR dataset in particular since the data was high-passed at 4Hz ARL used a kernel length of 32.
      F1, F2          : number of temporal filters (F1) and number of pointwise
                        filters (F2) to learn. Default: F1 = 8, F2 = F1 * D.
      D               : number of spatial filters to learn within each temporal
                        convolution. Default: D = 2
'''

class EEGNet(nn.Module):
    def __init__(self, nb_classes: int, Chans: int = 64, Samples: int = 128,
                 dropoutRate: float = 0.5, kernLength: int = 63,
                 F1:int = 8, D:int = 2):
        super().__init__()

        F2 = F1 * D

        # Make kernel size and odd number
        try:
            assert kernLength % 2 != 0
        except AssertionError:
            raise ValueError("ERROR: kernLength must be odd number")

        # In: (B, Chans, Samples, 1)
        # Out: (B, F1, Samples, 1)
        self.conv1 = nn.Conv1d(Chans, F1, kernLength, padding=(kernLength // 2))
        self.bn1 = nn.BatchNorm1d(F1) # (B, F1, Samples, 1)
        # In: (B, F1, Samples, 1)
        # Out: (B, F2, Samples - Chans + 1, 1)
        self.conv2 = nn.Conv1d(F1, F2, Chans, groups=F1)
        self.bn2 = nn.BatchNorm1d(F2) # (B, F2, Samples - Chans + 1, 1)
        # In: (B, F2, Samples - Chans + 1, 1)
        # Out: (B, F2, (Samples - Chans + 1) / 4, 1)
        self.avg_pool = nn.AvgPool1d(1)
        self.dropout = nn.Dropout(dropoutRate)

        # In: (B, F2, (Samples - Chans + 1) / 4, 1)
        # Out: (B, F2, (Samples - Chans + 1) / 4, 1)
        self.conv3 = SeparableConv1d(F2, F2, kernel_size=15, padding=7)
        self.bn3 = nn.BatchNorm1d(F2)
        # In: (B, F2, (Samples - Chans + 1) / 4, 1)
        # Out: (B, F2, (Samples - Chans + 1) / 32, 1)
        self.avg_pool2 = nn.AvgPool1d(1)
        # In: (B, F2 *  (Samples - Chans + 1) / 32)
        # 32x2960 (incomming) -> Outgoing = 1
        # self.fc = nn.Linear(F2 * ((Samples - Chans + 1) // 32), nb_classes)
        self.flatten = torch.flatten
        # self.fc = nn.Linear(2960, nb_classes)
        self.fc = nn.Linear(2992, nb_classes)


    def forward(self, x: torch.Tensor):
        # Block 1
        print("BLOCK 1")
        print("-------------------------")
        print()
        print("INPUT: ", x.shape)
        y1 = self.conv1(x)
        print("AFTER CONV1: ", y1.shape)
        y1 = self.bn1(y1)
        # print("AFTER BATCH1: ", y1.shape)
        y1 = self.conv2(y1)
        print("AFTER CONV2: ", y1.shape)
        y1 = F.relu(self.bn2(y1))
        print("AFTER RELU ACTIVATION: ", y1.shape)
        y1 = self.avg_pool(y1)
        print("AFTER AVEPOOL", y1.shape)
        y1 = self.dropout(y1)
        # print("AFTER DROPOUT: ", y1.shape)
        print()
        print("BLOCK 2")
        print("-------------------------")
        print()

        # Block 2
        y2 = self.conv3(y1)
        print("AFTER CONV3: ", y2.shape)
        y2 = F.relu(self.bn3(y2))
        print("AFTER RELU ACTIVATION: ", y2.shape)
        y2 = self.avg_pool2(y2)
        print("AFTER AVE2: ", y2.shape)
        y2 = self.dropout(y2)
        # print("dropout", y2.shape)
        y2 = self.flatten(y2, 1)
        print("AFTER FLATTEN: ", y2.shape)
        y2 = torch.sigmoid(self.fc(y2))
        print("fc", y2.shape)

        return y2

In [284]:
num_chans = 30

# if filter:
#     num_chans = len(filter)

model = EEGNet(Chans = num_chans, Samples=64, nb_classes=1, kernLength=7).to(device)

In [285]:
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

## Training

In [286]:
def binary_acc(y_pred, y_test):
    # print(f"Y pred: {y_pred}")
    # print(f"y_test: {y_test}")
    prediction = torch.round(y_pred)
    # print(f"Prediction: {prediction}")
    correct_pred = (prediction == y_test).float()
    acc = correct_pred.sum() / len(correct_pred)
    acc = torch.round(acc * 100)
    return acc

In [287]:
def train(model, optimizer, criterion, dataloader, epochs):
    start_epoch = 0

    best_model_wts = copy.deepcopy(model.state_dict())
    lowest_loss = 100

    accuracy_stats = {
        'train': [],
        "val": []
    }
    loss_stats = {
        'train': [],
        "val": []
    }

    check_path = glob.glob("./checkpoints/*.tar")
    if check_path:
        checkpoint = torch.load(check_path[0])
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        start_epoch = checkpoint['epoch']
        loss_stats = checkpoint['loss_stats']
        accuracy_stats = checkpoint['accuracy_stats']
        best_model_wts = checkpoint["best_wts"]
        lowest_loss = checkpoint["lowest_loss"]
        model.train()
        print(f"Found checkpoint. Epoch: {start_epoch-1} | Train acc: {accuracy_stats['train'][-1]} | Val acc: {accuracy_stats['val'][-1]}")


    for epoch in range(start_epoch, epochs):
        # Training
        train_loss = 0
        train_acc = 0
        data_size = 0
        iteration = 0
        for features, labels in dataloader:
            if iteration % 100 == 0:
                print(f"Iteration: {iteration} / {len(dataloader)}")
            iteration += 1
            features, labels = features.to(device), labels.to(device)
            features = features.float()
            labels = labels.float()
            print(features.shape)

            optimizer.zero_grad()
            # print(features.shape)
            # print(features)
            # print(model)
            y_pred = model(features)
            y_pred = torch.squeeze(y_pred)
            # print(y_pred)
            # print(labels)
            loss = criterion(y_pred, labels)
            acc = binary_acc(y_pred, labels)


            loss.backward()
            optimizer.step()
            train_loss += loss.item()
            train_acc += acc.item()

            # data_size += 1
            # # if data_size % 100 == 0:
            # print(f"{data_size} / {len(dataloader)}")
        
        # Validating
        with torch.no_grad():
            val_loss = 0
            val_acc = 0
            for val_features, val_labels in val_dataloader:
                val_features, val_labels = val_features.to(device), val_labels.to(device)
                # val_features = torch.unsqueeze(val_features, 1)
                val_features = val_features.float()
                val_labels = val_labels.float()


                val_pred = model(val_features)
                val_pred = torch.squeeze(val_pred)
                val_loss_item = criterion(val_pred, val_labels)
                val_acc_item = binary_acc(val_pred, val_labels)

                val_loss += val_loss_item.item()
                val_acc += val_acc_item.item()

                if val_loss < lowest_loss:
                    lowest_loss = val_loss  
                    best_model_wts = copy.deepcopy(model.state_dict())


        loss_stats['train'].append(train_loss/len(dataloader))
        loss_stats['val'].append(val_loss/len(val_dataloader))
        accuracy_stats['train'].append(train_acc/len(dataloader))
        accuracy_stats['val'].append(val_acc/len(val_dataloader))

        
        print(f'Epoch {epoch+0:03}: | Train Loss: {train_loss/len(dataloader):.5f} | Val Loss: {val_loss/len(val_dataloader):.5f} | Train Acc: {train_acc/len(dataloader):.3f} | Val Acc: {val_acc/len(val_dataloader):.3f}')
        path = f"./checkpoints/check_e.tar"
        torch.save({
            "epoch": (epoch+1),
            "model_state_dict": model.state_dict(),
            "optimizer_state_dict": optimizer.state_dict(),
            "loss_stats": loss_stats,
            "accuracy_stats": accuracy_stats,
            "best_wts": best_model_wts,
            "lowest_loss": lowest_loss,
        }, path)
    return best_model_wts, accuracy_stats, loss_stats

In [288]:
epochs = 15
# print(dataloader)
best_weights, accuracy_stats, loss_stats = train(model, optimizer, criterion, dataloader, epochs=epochs)

Iteration: 0 / 2151
torch.Size([1, 30, 128])
BLOCK 1
-------------------------

INPUT:  torch.Size([1, 30, 128])
AFTER CONV1:  torch.Size([1, 8, 128])
AFTER CONV2:  torch.Size([1, 16, 99])
AFTER RELU ACTIVATION:  torch.Size([1, 16, 99])
AFTER AVEPOOL torch.Size([1, 16, 99])

BLOCK 2
-------------------------

AFTER CONV3:  torch.Size([1, 16, 99])
AFTER RELU ACTIVATION:  torch.Size([1, 16, 99])
AFTER AVE2:  torch.Size([1, 16, 99])
AFTER FLATTEN:  torch.Size([1, 1584])


RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x1584 and 2992x1)